# Import Torch functions and tokenizers

In [1]:
import torch
from torch import Tensor, nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import dataset
from torch.utils.tensorboard import SummaryWriter

import regex as re
import os
import time
from tqdm import tqdm
import copy
import math

from model import TransformerModel
from utils import preProcessText, getTokenizer,try_gpu , word_piece_decoder, word_piece_encoder
from config import getConfig

import pickle

In [22]:
model_config, app_config = getConfig(small = True)
print(model_config)
print(app_config)

bptt=model_config["bptt"]
device = try_gpu(0)

{'emsize': 300, 'd_hid': 800, 'nlayers': 4, 'nhead': 4, 'dropout': 0.05, 'bptt': 32}
{'logs': 'tensorboard_logs', 'epochs': 10}


# Preprocessing Text

In [2]:
# t1[:5000]

In [3]:

file_path = 'data/preprocessed_morph.txt'
if not os.path.exists(file_path):
    print("Run morpheme_datagen notebook")
else:
    print(f"Reading file  : {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        text = preProcessText(text,tokenizer_type = 'morpheme')

Reading file  : data/preprocessed_morph.txt


In [7]:
text[:500]

'बर्दिबास * नगरपालिका को * तेस्रो नगर परिषदबाट पारित आव २०७३ * । ७४ को संशोधित र * २०७४ * । ७५ को प्रस्तावित नीति ? * कार्यक्रम * तथा * बजेट\nअार्थिक * वर्ष * २०७५७६ * काे नदिजन्य * पदार्थ काे * उत्खनन् गरी बिक्रि * वितरण * तथा अान्तरिक निकासी गर्ने * कार्य काे * बाेलपत्र सम्बन्धी * सुचना\nसक्ष ार सप्तरी अभियानमा सप्तरीबासी * सम्पूर्ण * सरोकारवालाहरु को * सहयोग र * सहभागिता का ो लागि अनुराोध छ । * सामुदायिक * अध्ययन * केन्द्र हरूको * नविकरण सम्बन्धमा । \nकाठमाडौं * ? १२ कातिक । * राष्ट्रपति * विद्या'

In [9]:
len(text.split('\n'))

123563

In [4]:
train_iter_first = text.split('\n')[:100000]
test_iter = text.split('\n')[100000:]

# Run Here

In [5]:
# import pickle


# tokenizer = get_tokenizer(None)

# vocab = build_vocab_from_iterator(
#     map(tokenizer, train_iter_first), specials=['<unk>'],max_tokens = 30000)
# vocab.set_default_index(vocab['<unk>'])


# # Save for first time
# with open('transformer_vocab_morpheme.pickle','wb') as f:
#     pickle.dump(vocab,f)

tokenizer,vocab = getTokenizer(tokenizer_type = 'morpheme')

# with open('tokenizers/transformer_vocab_morpheme.pickle','rb') as f:
#     vocab = pickle.load(f)

In [8]:
#dir(vocab)

In [7]:
#vocab.get_itos()

In [6]:
list(vocab.get_stoi().items())[:20]

[('गराउन', 630),
 ('तामाकोशी', 7912),
 ('धानको', 5649),
 ('कम्तीमा', 1348),
 ('सञ्चार', 1107),
 ('छिटै', 6908),
 ('आणविक', 5099),
 ('इलाका', 2201),
 ('राजपत्रमा', 13387),
 ('वर्षीय', 625),
 ('सम्भावना', 482),
 ('विकको', 11980),
 ('युनियनका', 11859),
 ('दावी', 1923),
 ('ठूलै', 4034),
 ('गरेका', 31),
 ('मैतीदेवी', 28939),
 ('हाँसो', 7269),
 ('अपरेटर', 13993),
 ('पासो', 19681)]

In [86]:
len(vocab)

30000

#  some utility functions

In [9]:
def split_list(l):
    splitted_list = []
    z = 0
    for i,idx in enumerate(l):
        if idx == 220:
            splitted_list.append(l[z:i])
            z = i+1
    if z <= len(l)-1:
        splitted_list.append(l[z:])
    return splitted_list

In [11]:
# k = split_list(l_)

In [10]:
def splits_to_token(splited_list):
    strings = [tokenizer.decode(l) for l in splited_list]
    
    return strings
    

In [152]:
# splits_to_token(k)

['महानायक राजेश हमाल अहिले चलचित्र क्षेत्रमा पातलिए ।']

In [156]:
# print(tokenizer.encode(' ').ids)

# tokenizer.decode([978,
#  261,
#  264,
# 624,
#  261,
#  263])

[624]


'하бд бг'

In [11]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long)
            for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
#     data = data.view(bsz, seq_len).t().contiguous()
    data = data.view(bsz,seq_len).t()
#     return data.to(device)
    return data


seq_length = 128
def get_batch(source: Tensor, i: int) -> tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(seq_length, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    #target = source[i+1:i+1+seq_len]
    return data, target

In [12]:
#Train and Test Split
train_data = data_process(train_iter_first)
test_data = data_process(test_iter)

In [13]:
len(train_data)

31139407

In [14]:
train_data[:40]

tensor([14037,     1,   146,     4,     1,   572,   778,     0,   985,  1879,
         1520,     1,     2,  3696,     4, 11933,     5,     1,   813,     1,
            2,  1527,     4,  4469,   366,     3,     1,    71,     1,    20,
            1,   403, 25615,     1,    64,     1,  5413,     1,  1624,     0])

# Working with a dummy Sample

In [17]:
#Sample Data


text = ['आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य']
#text = ['जनसंख्या']
sample_data = data_process(
    text)

In [18]:
sample_data

tensor([2365, 9109,  666,    0, 1027,    0,    0,  422,  388,  474, 7705,  315,
           0,  422,  388])

In [19]:
sample_data = batchify(sample_data, 2)
print("Given word:", text[0])
sample_data

Given word: आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य


tensor([[2365,  422],
        [9109,  388],
        [ 666,  474],
        [   0, 7705],
        [1027,  315],
        [   0,    0],
        [   0,  422]])

# Model Definition

In [23]:
batched_train_data = batchify(train_data, bptt).to(device)  # shape [seq_len, batch_size]
batched_test_data = batchify(test_data, bptt).to(device)

In [24]:
def get_model(model_config, ntokens):
    emsize = model_config["emsize"]
    d_hid = model_config["d_hid"]
    nlayers = model_config["nlayers"]
    nhead = model_config["nhead"]
    dropout = model_config["dropout"]
    model = TransformerModel(ntokens, emsize,nhead, d_hid, nlayers, dropout)
    return model

In [25]:
ntokens = len(vocab)
model = get_model(model_config, ntokens).to(device)
torch.cuda.memory_allocated()

401597952

In [21]:
batched_train_data = batchify(train_data, bptt)  # shape [seq_len, batch_size]
batched_test_data = batchify(test_data, bptt)

In [107]:
seq_length = 128

# Hyper-Parameter Tuning

In [26]:
criterion = nn.CrossEntropyLoss()
lr = 1  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
softmax = nn.Softmax(dim=2)
#softmax = nn.LogSoftmax(dim=2)

In [36]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [37]:
def train(model: nn.Module) -> None:
    global epoch
    global global_step
    model.train()  # turn on train mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(batched_train_data) // bptt
    progress_bar = tqdm(enumerate(range(0, batched_train_data.size(0) - 1, bptt)), total=num_batches, desc=f'Epoch {epoch}', ncols=80)
    for batch_idx, i in progress_bar:
        ### batch_idx -> (1, 2, 3, 4, ...)
        ### i -> (0, bptt, 2*bptt, ....)
        data, targets = get_batch(batched_train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()

        ## calculate the postfix description for the progress bar
        cur_loss = total_loss / (batch_idx + 1)
        ppl = math.exp(cur_loss)
        
        progress_bar.set_postfix({"loss": cur_loss, "ppl" : ppl}, refresh=True)
        
        writer.add_scalar('loss/train loss', cur_loss, global_step)
        writer.flush()
        writer.add_scalar('ppl/train perplexity', ppl, global_step)
        writer.flush()
        global_step += 1

In [38]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(eval_data) // bptt
    with torch.no_grad():
        progress_bar = tqdm(enumerate(range(0, eval_data.size(0) - 1, bptt)), total=num_batches, desc=f'Validation {epoch}', ncols=80)
        for batch_idx, i in progress_bar:
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_softmax = softmax(output)
            output_softmax_permuted = output_softmax.permute(1, 0, 2)
            indices = torch.argmax(output_softmax_permuted, dim=2)
            target_indices = targets.t()
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    
    eval_loss = total_loss / (len(eval_data) - 1)
    eval_ppl = math.exp(eval_loss)

    writer.add_scalar('loss/val loss', eval_loss, global_step)
    writer.flush()
    writer.add_scalar('ppl/val perplexity', eval_ppl, global_step)
    writer.flush()

    return eval_loss

In [39]:
softmax = nn.Softmax(dim=2)
#softmax = nn.LogSoftmax(dim=2)

# Training Data

In [40]:
best_model_path = 'models/best_model_mp.pt'

In [ ]:
# Loop over epochs. Save the model if the validation loss is the best
# we've seen so far. Adjust the learning rate after each epoch.
best_val_loss = float('inf')
initial_epoch = 0
epochs = app_config["epochs"]
global_step = 0
best_model = None

# preload the model if exists to train more epochs

if os.path.exists(best_model_path):
    print(f"Preloading model {best_model_path}")
    state = torch.load(best_model_path)
    
    initial_epoch = state['epoch'] + 1
    model.load_state_dict(state['model_state_dict'])
    optimizer.load_state_dict(state['optimizer_state_dict'])
    global_step = state['global_step']
    best_val_loss = state['best_val_loss']
    
    print(initial_epoch, global_step, best_val_loss)

# initializing the tensorbaord log writer
writer = SummaryWriter(app_config["logs"])


for epoch in range(initial_epoch, epochs):
    train(model)
    eval_loss = evaluate(model, batched_test_data)

    # save the model if validation loss decreases

    if eval_loss < best_val_loss:
        print(f"eval perplexity : {math.exp(eval_loss)}")
        print("saving the model")
        best_val_loss = eval_loss
        best_model = copy.deepcopy(model)

        directory_path = 'models'
        # Create the directory if it doesn't exist
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
        torch.save({
                'epoch': epoch,
                'model_state_dict': best_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'global_step': global_step, 
                'best_val_loss' : best_val_loss,
            }, os.path.join(directory_path, 'best_model_wp.pt'))

Epoch 0:   1%|         | 405/30409 [01:19<1:35:59,  5.21it/s, loss=6.9, ppl=988]

In [131]:
lnsoftmax = nn.LogSoftmax(dim=2)

# Data Generation

In [ ]:
def loadModel(best_model_path):
    global model
    if os.path.exists(best_model_path):
        print(f"Preloading model {best_model_path}")
        if torch.cuda.is_available():
            state = torch.load(best_model_path)
        else:
            state = torch.load(best_model_path, map_location=torch.device('cpu'))
        model.load_state_dict(state['model_state_dict'])
        return model
    else:
        raise Exception("Model Not Found")
        
loaded_model = loadModel(best_model_path)

In [29]:

def generator(model: nn.Module, gen_data: Tensor, no_words = 10):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        else:
            src_mask_ = src_mask[:,:]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.argmax(output_softmax_permuted, dim=2)
        pred_text.append([vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        if(batch_size < 16):
            gen_data = torch.cat((gen_data[:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
            
    return pred_text





def nonnaive_generator(model: nn.Module, gen_data: Tensor, no_words = 5,k=50):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):

        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.topk(output_softmax_permuted,k ,dim=2).indices.squeeze(0)
        
        values = torch.topk(softmax(output_softmax_permuted),k ,dim=2).values
        values = values/torch.sum(values,dim = 2,keepdims = True)

        
        ind_sampled = torch.distributions.Categorical(values.squeeze(0)).sample()
        next_index = indices[-1][ind_sampled[-1]]
        

        pred_text.append([vocab.lookup_token(next_index)][0])
        if(batch_size < 15):
            gen_data = torch.cat((gen_data[:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
            
    return pred_text

In [200]:
import morfessor
import math

with open('models/morfessor_model.p','rb') as f:
    models = pickle.load(f)

In [240]:
def convert_to_morph(l):
    a = []
    for v in l:
        t1 = '-'.join(models[0].viterbi_segment(v)[0])
        tr = re.sub(r'[ ]+', r' ', t1)
        tr = re.sub(r'- -', r'*', tr)
        tr = re.sub(r'-[ ]+', r'*', tr)
        tr = re.sub(r'[ ]+-', r'*',tr)
        tr = re.sub(r' ', r' * ',tr)
        tr = re.sub(r'\*', r' * ',tr)
        tr = re.sub(r'  ', r' ', tr)
        tr = re.sub(r'-', r' ',tr)
        a.append(tr)
    return a

def revert_sentence(text):
#     tr = re.sub(r' ', r'',text)
    tr = re.sub(r'\*', r' ',text)
    return tr

In [257]:
# st = ['लामो समयसम्म प्रयोग गर्न सकिन्छ ।']
# st = ['तपाईंलाई कस्तो पुस्तकहरू मन']
st = ['नेपालमा आधुनिक']
st = convert_to_morph(st)
st_i = data_process(st)
st_i = st_i.unsqueeze(1).to(device)

In [141]:
bptt = 32

In [235]:
z = nonnaive_generator(best_model, st_i,no_words = 100)

i: 0
next word:  *
0 Values:  tensor([9.9716e-01, 3.5321e-04, 3.4179e-04, 3.4124e-04, 3.0203e-04, 2.6042e-04,
        1.5666e-04, 8.9967e-05, 8.1885e-05, 7.5854e-05, 5.7456e-05, 5.2657e-05,
        5.0898e-05, 4.1669e-05, 3.7929e-05, 3.4033e-05, 3.3421e-05, 3.1964e-05,
        3.0127e-05, 2.8189e-05, 2.4490e-05, 2.1524e-05, 2.0009e-05, 1.9502e-05,
        1.9442e-05, 1.8965e-05, 1.8100e-05, 1.7837e-05, 1.6394e-05, 1.6279e-05,
        1.6151e-05, 1.5967e-05, 1.5217e-05, 1.4853e-05, 1.4570e-05, 1.4037e-05,
        1.3350e-05, 1.3029e-05, 1.3027e-05, 1.2645e-05, 1.1703e-05, 1.1530e-05,
        1.1190e-05, 1.1017e-05, 1.0456e-05, 1.0316e-05, 1.0081e-05, 9.7230e-06,
        9.6726e-06, 9.0291e-06], device='cuda:0', grad_fn=<SelectBackward0>) Gen_data:  tensor([[ 69],
        [  1],
        [219],
        [  1],
        [526],
        [  1],
        [ 18]], device='cuda:0') possible tokens:  tensor([   1,    7,   11,   10,    4,    8,    0,   26,  718,   89,   53,  136,
         108,   47,  

8 Values:  tensor([9.9912e-01, 1.8030e-04, 1.2624e-04, 1.0364e-04, 8.4317e-05, 5.3482e-05,
        3.1102e-05, 3.0312e-05, 2.2118e-05, 2.1373e-05, 1.9640e-05, 1.9611e-05,
        1.6902e-05, 1.3005e-05, 1.1351e-05, 1.1018e-05, 9.0944e-06, 8.4157e-06,
        7.1925e-06, 7.1156e-06, 6.4921e-06, 5.9643e-06, 5.6695e-06, 5.1994e-06,
        4.8299e-06, 4.3233e-06, 4.2809e-06, 3.9767e-06, 3.8612e-06, 3.6409e-06,
        3.5611e-06, 3.5429e-06, 3.3863e-06, 3.2156e-06, 2.9770e-06, 2.9640e-06,
        2.9008e-06, 2.5771e-06, 2.5132e-06, 2.4295e-06, 2.4142e-06, 2.4048e-06,
        2.2938e-06, 2.1803e-06, 2.1793e-06, 2.0875e-06, 2.0659e-06, 2.0644e-06,
        2.0427e-06, 1.9776e-06], device='cuda:0', grad_fn=<SelectBackward0>) Gen_data:  tensor([[ 69],
        [  1],
        [219],
        [  1],
        [526],
        [  1],
        [ 18],
        [  1],
        [216],
        [  1],
        [  2],
        [  1],
        [165],
        [  1],
        [  3]], device='cuda:0') possible tokens:  

next word:  *
18 Values:  tensor([9.5596e-01, 7.4406e-03, 5.4356e-03, 4.8867e-03, 4.8567e-03, 4.5013e-03,
        3.9231e-03, 1.0253e-03, 9.9877e-04, 8.1819e-04, 7.9589e-04, 7.2469e-04,
        6.9461e-04, 5.7685e-04, 4.4743e-04, 4.1816e-04, 3.1889e-04, 3.1077e-04,
        3.0748e-04, 2.9646e-04, 2.5897e-04, 2.5568e-04, 2.2791e-04, 2.2650e-04,
        2.2250e-04, 2.1890e-04, 2.1752e-04, 2.0915e-04, 2.0613e-04, 2.0435e-04,
        1.9515e-04, 1.9512e-04, 1.9378e-04, 1.9016e-04, 1.8494e-04, 1.5999e-04,
        1.5890e-04, 1.5481e-04, 1.5401e-04, 1.5107e-04, 1.4213e-04, 1.3511e-04,
        1.3312e-04, 1.3088e-04, 1.2948e-04, 1.2720e-04, 1.2238e-04, 1.2160e-04,
        1.1893e-04, 1.1856e-04], device='cuda:0', grad_fn=<SelectBackward0>) Gen_data:  tensor([[  2],
        [  1],
        [165],
        [  1],
        [  3],
        [  1],
        [  0],
        [  1],
        [119],
        [  1],
        [ 51],
        [  1],
        [  9],
        [  1],
        [  0]], device='cuda:0') pos

        [  1]], device='cuda:0') possible tokens:  tensor([  0,   9,  51,  15,  36,  61,  95,  41, 157,  50,  54,  60, 105,  17,
         45,  23, 121, 147,   3,  39, 289,  24, 117,  49, 100,  96,  48,  63,
        153,  73,  69, 277,  91, 163, 169,  43, 168, 187, 104, 141,  66, 138,
          5, 212,  30, 112,  37,  74, 102, 479], device='cuda:0') Pred_data:  tensor(0, device='cuda:0')
i: 36
next word:  *
36 Values:  tensor([9.6228e-01, 6.7142e-03, 5.4245e-03, 4.6066e-03, 4.0920e-03, 3.9682e-03,
        2.7827e-03, 9.3287e-04, 7.7606e-04, 6.4588e-04, 6.0124e-04, 5.7726e-04,
        4.8801e-04, 4.2028e-04, 3.1469e-04, 3.0790e-04, 2.9757e-04, 2.7209e-04,
        2.2054e-04, 2.1888e-04, 2.1314e-04, 2.1019e-04, 2.0738e-04, 1.9908e-04,
        1.9049e-04, 1.7512e-04, 1.7496e-04, 1.6488e-04, 1.5580e-04, 1.5524e-04,
        1.4741e-04, 1.4037e-04, 1.3596e-04, 1.3111e-04, 1.2038e-04, 1.2011e-04,
        1.1860e-04, 1.1522e-04, 1.0802e-04, 1.0575e-04, 1.0573e-04, 1.0452e-04,
        1.0434e-04

next word:  त्यही
57 Values:  tensor([0.2045, 0.0769, 0.0518, 0.0483, 0.0426, 0.0381, 0.0277, 0.0273, 0.0237,
        0.0233, 0.0222, 0.0219, 0.0180, 0.0170, 0.0167, 0.0164, 0.0163, 0.0163,
        0.0160, 0.0158, 0.0141, 0.0138, 0.0136, 0.0133, 0.0131, 0.0104, 0.0097,
        0.0094, 0.0093, 0.0092, 0.0088, 0.0086, 0.0086, 0.0082, 0.0082, 0.0080,
        0.0078, 0.0077, 0.0075, 0.0075, 0.0072, 0.0071, 0.0070, 0.0062, 0.0060,
        0.0060, 0.0059, 0.0058, 0.0058, 0.0057], device='cuda:0',
       grad_fn=<SelectBackward0>) Gen_data:  tensor([[ 1],
        [73],
        [ 1],
        [ 9],
        [ 1],
        [41],
        [ 1],
        [41],
        [ 1],
        [ 0],
        [ 1],
        [ 2],
        [ 1],
        [ 3],
        [ 1]], device='cuda:0') possible tokens:  tensor([  0,  27,  73,   3,  69, 169, 252,  22, 168, 141, 528,  96, 165,   5,
        104,  65,  37, 177,   2, 283, 123,   1,  54, 120, 138, 349, 555, 246,
        451,  45, 615, 591,  95, 201, 443, 354,  41, 210,

78 Values:  tensor([9.9373e-01, 8.0981e-04, 6.3579e-04, 5.3420e-04, 3.8081e-04, 3.6821e-04,
        2.9310e-04, 2.9218e-04, 2.8441e-04, 1.3960e-04, 1.2665e-04, 1.1927e-04,
        1.0601e-04, 1.0006e-04, 9.8885e-05, 9.1651e-05, 8.3943e-05, 8.2076e-05,
        8.1881e-05, 7.9215e-05, 7.7695e-05, 7.6518e-05, 7.4331e-05, 6.7175e-05,
        6.5060e-05, 6.4153e-05, 6.1630e-05, 6.1066e-05, 5.6048e-05, 5.3963e-05,
        5.3811e-05, 5.3726e-05, 5.2732e-05, 5.2283e-05, 5.2097e-05, 5.1425e-05,
        5.1201e-05, 4.8892e-05, 4.7421e-05, 4.6260e-05, 4.3426e-05, 4.3249e-05,
        4.2365e-05, 4.1916e-05, 4.0729e-05, 3.7333e-05, 3.7236e-05, 3.7030e-05,
        3.6618e-05, 3.5978e-05], device='cuda:0', grad_fn=<SelectBackward0>) Gen_data:  tensor([[   3],
        [   1],
        [  37],
        [   1],
        [  17],
        [   1],
        [   3],
        [   1],
        [1222],
        [   1],
        [   0],
        [   1],
        [   0],
        [   1],
        [  34]], device='cuda:0') po

99 Values:  tensor([9.5173e-01, 1.0498e-02, 6.2747e-03, 6.0837e-03, 4.4324e-03, 4.3379e-03,
        4.2110e-03, 9.0603e-04, 7.8384e-04, 7.7050e-04, 7.4400e-04, 6.8752e-04,
        6.7869e-04, 5.9368e-04, 4.8990e-04, 3.4312e-04, 3.2031e-04, 3.1606e-04,
        2.5219e-04, 2.5133e-04, 2.4423e-04, 2.4261e-04, 2.3338e-04, 2.2889e-04,
        2.2426e-04, 2.2154e-04, 2.1885e-04, 2.1202e-04, 2.1082e-04, 1.9683e-04,
        1.9369e-04, 1.8903e-04, 1.8839e-04, 1.8826e-04, 1.8760e-04, 1.8028e-04,
        1.7980e-04, 1.7915e-04, 1.5464e-04, 1.4568e-04, 1.3602e-04, 1.3589e-04,
        1.3351e-04, 1.3274e-04, 1.3170e-04, 1.2871e-04, 1.2208e-04, 1.2126e-04,
        1.2000e-04, 1.1761e-04], device='cuda:0', grad_fn=<SelectBackward0>) Gen_data:  tensor([[  1],
        [  3],
        [  1],
        [  0],
        [  1],
        [  3],
        [  1],
        [165],
        [  1],
        [ 45],
        [  1],
        [121],
        [253],
        [  1],
        [  0]], device='cuda:0') possible tokens: 

In [241]:
revert_sentence(''.join(z__))

' सकिन्छ । अनि ? <unk> भनेर कुरा पनि <unk> थिएन ? उनले भने । म अब यसको <unk> हो । तर त्यो पनि त त <unk> । ? त्यही वा मन ? उनले भने ? त्यसले <unk> <unk> हुन्छ भने कुनै <unk> ? <unk> ? अनि केही प्रयोगहरुको <unk> '